<h1>Max-cut as IP instance</h1>
<h3>Import libraries</h3>

In [12]:
import numpy as np
import networkx as nx
import networkx.drawing
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

%matplotlib

Using matplotlib backend: Qt5Agg


<h3>Define functions</h3>

In [41]:
def PlotGraphWithEdges(G):
    pos = nx.spring_layout(G)
    # Draw the graph according to node positions
    nx.draw(G, pos, with_labels=True)
    # Create edge labels
    labels = nx.get_edge_attributes(G,'weight')

    # Draw edge labels according to node positions
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)

    plt.show()
    
def printSolution(m):
    if m.status == GRB.OPTIMAL:
        print('\nCost: %g' % m.objVal)
    else:
        print('No solution')

In [14]:
# Number of nodes
n = 7

# Max weight
w_max = 10

# Create random integer matrix
W = np.random.randint(w_max, size=(n, n))

# Set diagonal to 0
np.fill_diagonal(W, 0)

# Make it symmetric
W = (W + W.T)/2

# Convert matrix to graph
G = nx.from_numpy_matrix(W)

In [15]:
# Draw
PlotGraphWithEdges(G)

In [49]:
# Define x variables
v_label = 'v'
v_labels = []
for i in range(n):
    v_labels.append(v_label + str(i))

# Define gurobi model
m = gp.Model("Max-Cut")

v = m.addVars(n, lb=-1, ub=1, vtype=GRB.INTEGER, name="v")

# Create weight pairs
weights = {}
for i in range(n):
    for j in range(n):
        weights.update({(v_label + str(i), v_label + str(j)): W[i,j]})
        
m.setObjective(gp.quicksum(0.5*W[i,j]*(1-v[i]*v[j]) for i in range(n) for j in range(n)), GRB.MAXIMIZE)

m.addConstrs((v[i]*v[j]>=1 for i in range(n) for j in range(i,n)), "Separation")
m.write()
#m.optimize()

#printSolution(m)

TypeError: write() takes exactly 2 positional arguments (1 given)

In [20]:
print(weights)

{('v0', 'v0'): 0.0, ('v0', 'v1'): 2.0, ('v0', 'v2'): 4.5, ('v0', 'v3'): 2.0, ('v0', 'v4'): 3.5, ('v0', 'v5'): 8.5, ('v0', 'v6'): 6.0, ('v1', 'v0'): 2.0, ('v1', 'v1'): 0.0, ('v1', 'v2'): 2.5, ('v1', 'v3'): 6.5, ('v1', 'v4'): 5.0, ('v1', 'v5'): 4.5, ('v1', 'v6'): 1.0, ('v2', 'v0'): 4.5, ('v2', 'v1'): 2.5, ('v2', 'v2'): 0.0, ('v2', 'v3'): 8.5, ('v2', 'v4'): 4.0, ('v2', 'v5'): 1.5, ('v2', 'v6'): 3.5, ('v3', 'v0'): 2.0, ('v3', 'v1'): 6.5, ('v3', 'v2'): 8.5, ('v3', 'v3'): 0.0, ('v3', 'v4'): 1.5, ('v3', 'v5'): 5.5, ('v3', 'v6'): 6.5, ('v4', 'v0'): 3.5, ('v4', 'v1'): 5.0, ('v4', 'v2'): 4.0, ('v4', 'v3'): 1.5, ('v4', 'v4'): 0.0, ('v4', 'v5'): 1.0, ('v4', 'v6'): 8.0, ('v5', 'v0'): 8.5, ('v5', 'v1'): 4.5, ('v5', 'v2'): 1.5, ('v5', 'v3'): 5.5, ('v5', 'v4'): 1.0, ('v5', 'v5'): 0.0, ('v5', 'v6'): 2.0, ('v6', 'v0'): 6.0, ('v6', 'v1'): 1.0, ('v6', 'v2'): 3.5, ('v6', 'v3'): 6.5, ('v6', 'v4'): 8.0, ('v6', 'v5'): 2.0, ('v6', 'v6'): 0.0}
